In [ ]:
import pandas as pd
import numpy as np

import os
import sys


import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio


import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint


In [ ]:
# Paths for data.
Ravdess = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
Ravdess_song = "/kaggle/input/ravdess-emotional-song-audio/audio_song_actors_01-24/"


In [ ]:
ravdess_directory_list = os.listdir(Ravdess)
file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are different directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()




In [ ]:
ravdess_directory_list_song = os.listdir(Ravdess_song)
file_emotion_song = []
file_path_song = []
for dir in ravdess_directory_list_song:
    # as their are different directory we need to extract files for each actor.
    actor = os.listdir(Ravdess_song + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion_song.append(int(part[2]))
        file_path_song.append(Ravdess_song + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df_song = pd.DataFrame(file_emotion_song, columns=['Emotions'])

# dataframe for path of files.
path_df_song = pd.DataFrame(file_path_song, columns=['Path'])
Ravdess_df_song = pd.concat([emotion_df_song, path_df_song], axis=1)

# changing integers to actual emotions.
Ravdess_df_song.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df_song.describe()





In [ ]:
Ravdess_df = pd.concat([Ravdess_df,Ravdess_df_song])

Ravdess_df.describe()

In [ ]:
data_path = Ravdess_df
data_path.to_csv("data_path.csv",index=False)
data_path.info()

In [ ]:
plt.title('Count of Emotions', size=16)
sns.countplot(data_path.Emotions)
plt.ylabel('Count', size=12)
plt.xlabel('Emotions', size=12)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.show()


In [ ]:
#checking for the data
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)

In [ ]:
#visualizing a single audio file
plt.figure(figsize=(14,4))
librosa.display.waveplot(y=data, sr=sample_rate)


In [ ]:
#DATA AUGMENTATION

def noise(data):
    noise_amp = 0.025*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.6):
    return librosa.effects.time_stretch(data, rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)


# taking any example and checking for techniques.
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)





In [ ]:
 def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally matlab equivalent result = [result zcr]

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    # tonnetz
    tonn = np.mean(librosa.feature.tonnetz(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, tonn)) #stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    
    
    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2)) # stacking vertically
    
    # data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch)
    result = np.vstack((result, res3)) # stacking vertically
    
    return result # +1440*3 is our new number of data (number of rows) and columns are 6 features extracted.

In [ ]:
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)

zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
stft = np.abs(librosa.stft(data))
chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)

feature = get_features(path)

feature.shape




In [ ]:
path = np.array(data_path.Path)[1]
path

In [ ]:
#checking the shapes of the features
zcr.shape
chroma_stft.shape
mfcc.shape
mfcc.shape
mel.shape

In [ ]:
X, Y = [], []
for path, emotion in zip(data_path.Path, data_path.Emotions):
    feature = get_features(path) # outputs result matrix (1data*3) * (6)
    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [ ]:
len(X), len(Y), data_path.Path.shape


In [ ]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv('features.csv', index=False)
Features.head()

In [ ]:
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

# one hot encoding our y
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
#train test split

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
#normalizes the data by -miu/std.dev
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
from numpy import savetxt

savetxt('xtest.csv', x_test, delimiter=',')
savetxt('xtrain.csv', x_train, delimiter = ',')

savetxt('ytest.csv', y_test, delimiter=',')
savetxt('ytrain.csv', y_train, delimiter = ',')

In [ ]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
#the convolutional neural network that we are going to use

model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=8, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

In [ ]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=3, min_lr== 0.0000001)
history=model.fit(x_train, y_train, batch_size=256, epochs=90, validation_data=(x_test, y_test), callbacks=[rlrp])

In [ ]:
model.save_weights("312weight.h5")

In [ ]:
model.save('./')

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
print("Accuracy on validation : " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
#visual aid of how the metrics vary with accuracy
#plotting the losses

epochs = [i for i in range(100)]
fig , axis = plt.subplots(1,1)


train_loss = history.history['loss']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
axis.plot(epochs , train_loss , label = 'Training Loss')
axis.plot(epochs , test_loss , label = 'Testing Loss')
axis.set_title('Training & Testing Loss')
axis.legend()
axis.set_xlabel("Epochs")

plt.savefig('Losses.png')

In [ ]:
#plotting the accuracy of the training and the validation set
epochs = [i for i in range(100)]
fig , axis = plt.subplots(1,1)

train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

fig.set_size_inches(20,10)
axis.plot(epochs , train_accuracy , label = 'Training Accuracy')
axis.plot(epochs , test_accuracy , label = 'Testing Accuracy')
axis.set_title('Training & Testing Accuracy')
axis.legend()
axis.set_xlabel("Epochs")
plt.show()

plt.savefig('Accuracy_.png')

In [ ]:
#plotting the accuracy of the training and the validation set
epochs = [i for i in range(100)]
fig , axis = plt.subplots(1,1)

train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

fig.set_size_inches(20,6)
axis.plot(epochs , train_accuracy , label = 'Training Accuracy')
axis.plot(epochs , test_accuracy , label = 'Testing Accuracy')
axis.set_title('Training & Testing Accuracy')
axis.legend()
axis.set_xlabel("Epochs")
plt.show()

plt.savefig('Accuracies.png')

In [ ]:
 # Loads the weights #for pretrained weights see our directory
    
model.load_weights('../input/weightsss/312weight.h5')

# Re-evaluate the model
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))